In [1]:
#pip install --upgrade boto3

In [2]:
import zipfile,subprocess,shutil
import json,time,os,sys
import boto3 

In [3]:
#############################################################################
#############################################################################

In [4]:
# Specify your AWS credentials directly (avoid this in production; use IAM roles or environment variables)
aws_access_key_id     = 'blablabla'
aws_secret_access_key = 'blablabla'
aws_region            = 'us-east-1' 
aws_role  =   'arn:aws:iam::1111111111:role/misc-admin-role' 

In [5]:
#############################################################################
#############################################################################

In [6]:
#-------------------------------------------------------------

FP, FN = 'FILES/','lambda_func.py'   #<<<<<<<<<<<<<<<<<<<<<<<<
lambda_func  = 'JamesTestFuncZ'      #<<<<<<<<<<<<<<<<<<<<<<<< 

#-------------------------------------------------------------

LamFuncCode = '''

def lambda_handler(event, context):
    return {
        'statusCode': 200,
        'body': 'Hello from Lambda! My name is James!'
    }
'''

#-------------------------------------------------------------

handler_name = FN.replace('.py','.lambda_handler') 

with open(FP+FN, 'w') as file: 
    file.write(LamFuncCode)

# Define the file name and path
source_file = FP+FN
zip_file    = source_file.replace('.py','.zip') 

# Create a new zip file and add the source file to it
with zipfile.ZipFile(zip_file, 'w') as zipf: 
    zipf.write(source_file, arcname=FN)  
    
#------------------------------------------------------------- 

In [7]:
#############################################################################
#############################################################################

In [8]:
# Initialize a boto3 client
lambda_client = boto3.client(
    'lambda',
    region_name           = aws_region,
    aws_access_key_id     = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key
) 

# Path to the ZIP file containing your Lambda function code
zip_file_path = str(zip_file) 

# Read the zip file as binary code to upload
with open(zip_file_path, 'rb') as f:
    zipped_code = f.read()

# Create the Lambda function
response = lambda_client.create_function(
    FunctionName = lambda_func, Runtime = 'python3.8', Role = aws_role,
    Handler     = handler_name,  
    Code        = dict(ZipFile=zipped_code),
    Description = 'A sample Lambda function. - James',
    Timeout     = 30,      # Maximum allowable timeout
    MemorySize  = 128,  # Allocated memory in MB       
)

print('Done.') 

Done.


In [9]:
print(response) 

{'ResponseMetadata': {'RequestId': '81c1a4bf-fb17-42e9-9909-fa80637e31ac', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 15 May 2024 21:10:04 GMT', 'content-type': 'application/json', 'content-length': '1358', 'connection': 'keep-alive', 'x-amzn-requestid': '81c1a4bf-fb17-42e9-9909-fa80637e31ac'}, 'RetryAttempts': 0}, 'FunctionName': 'JamesTestFuncZ', 'FunctionArn': 'arn:aws:lambda:us-east-1:590183695003:function:JamesTestFuncZ', 'Runtime': 'python3.8', 'Role': 'arn:aws:iam::590183695003:role/james-admin-role', 'Handler': 'lambda_func.lambda_handler', 'CodeSize': 265, 'Description': 'A sample Lambda function. - James', 'Timeout': 30, 'MemorySize': 128, 'LastModified': '2024-05-15T21:10:04.625+0000', 'CodeSha256': 'kyu8ZzNKOSi56kQbi/T+gxUZirh4bg3FgZub/BqZJrA=', 'Version': '$LATEST', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '58d0785d-d32f-4138-a830-4a45022a86db', 'State': 'Pending', 'StateReason': 'The function is being created.', 'StateReasonCode': 'Creating', 'Pac

In [10]:
#############################################################################
#############################################################################

In [11]:
# Initialize a boto3 client for Lambda
lambda_client = boto3.client(
    'lambda',
    region_name           = aws_region,
    aws_access_key_id     = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key
) 

# List all Lambda functions
response = lambda_client.list_functions()

# Iterate through the list of functions and print their details
for function in response['Functions']:
    print(f"Function Name: {function['FunctionName']}")
    print(f"Runtime: {function['Runtime']}")
    print(f"Description: {function.get('Description', 'No description provided')}")
    print(f"Last Modified: {function['LastModified']}")
    print(f"Memory Size: {function['MemorySize']} MB") 
    print(f"Timeout: {function['Timeout']} seconds")
    print("-" * 40)  

Function Name: JamesTestFunc1
Runtime: python3.8
Description: 
Last Modified: 2024-05-15T01:11:43.000+0000
Memory Size: 128 MB
Timeout: 3 seconds
----------------------------------------
Function Name: JamesTestFunc2
Runtime: python3.8
Description: A sample Lambda function. - James
Last Modified: 2024-05-15T02:15:19.130+0000
Memory Size: 128 MB
Timeout: 30 seconds
----------------------------------------
Function Name: JamesTestFuncZ
Runtime: python3.8
Description: A sample Lambda function. - James
Last Modified: 2024-05-15T21:10:04.625+0000
Memory Size: 128 MB
Timeout: 30 seconds
----------------------------------------
Function Name: JamesTestFunc3
Runtime: python3.8
Description: A sample Lambda function. - James
Last Modified: 2024-05-15T02:59:32.690+0000
Memory Size: 128 MB
Timeout: 30 seconds
----------------------------------------


In [12]:
#############################################################################
#############################################################################

In [13]:
NOTES = '''

### HOW TO DELETE LAMBDA FUNCTIONS:

# List of Lambda functions to delete
functions_to_delete = ['JamesTestFunc2', 'JamesTestFunc3']

lambda_client = boto3.client(
    'lambda',
    region_name           = aws_region,
    aws_access_key_id     = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key
)  

# Iterate through the list and delete each function
for func_name in functions_to_delete:
    try:
        response = lambda_client.delete_function(FunctionName=func_name)
        print(f"Successfully deleted {func_name}")
    except Exception as e:
        print(f"Failed to delete {func_name}: {str(e)}")
        
# Successfully deleted JamesTestFunc2
# Successfully deleted JamesTestFunc3 

'''

In [14]:
#############################################################################
#############################################################################

In [15]:
time.sleep(5) # Sleep for 5 seconds  

In [16]:
### HOW TO CALL THE EXISTING LAMBDA FUNCTIONS: 

lambda_client = boto3.client(
    'lambda',
    region_name           = aws_region,
    aws_access_key_id     = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key
)  

# Prepare your payload (ensure this matches what your Lambda function expects)
payload = json.dumps({
    "key1": "value1",
    "key2": "value2"
})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName   = lambda_func,        # Replace with your Lambda function's name
    InvocationType = 'RequestResponse',  # Use 'Event' for asynchronous execution
    Payload = payload
) 

# Parse the response payload
response_payload = json.loads(response['Payload'].read())  

In [17]:
# Print the response
print(response_payload) 

{'statusCode': 200, 'body': 'Hello from Lambda! My name is James!'}


In [ ]:
#############################################################################
#############################################################################

In [ ]:
#############################################################################
#############################################################################

In [18]:
#.